**Лабораторная работа 4**

**Хаммуд Хала ИУ5И-21М**

**Реализация алгоритма Policy Iteration**

**Задание:** На основе рассмотренного на лекции примера реализуйте алгоритм Policy Iteration для любой среды обучения с подкреплением (кроме рассмотренной на лекции среды Toy Text / Frozen Lake) из библиотеки Gym (или аналогичной библиотеки).

**Описание задания**

В рамках лабораторной работы №4 по теме «Обучение на основе алгоритма Policy Iteration» необходимо реализовать базовый метод обучения с подкреплением — итерацию стратегии (policy iteration) — в символьной среде. В качестве среды используется Taxi-v3 из библиотеки Gymnasium, которая имитирует дискретную модель управления действиями агента в ограниченном пространстве.

Особенностью данной реализации является интерпретация среды как условной модели поведения человека с разным уровнем усталости. Агент, построенный в ходе работы, получает и обновляет вероятностную стратегию выбора действий в зависимости от состояния, имитируя когнитивную активность утомлённого или внимательного пользователя.

Цель задания —:

реализовать алгоритм итерации стратегии (policy evaluation и policy improvement);

обучить агента на основе среды Taxi-v3;

интерпретировать полученную стратегию как поведение человека с учётом уровня усталости;

визуализировать или вывести стратегию агента после обучения;

при возможности — продемонстрировать поведение обученного агента.



In [36]:
!pip install gymnasium

In [37]:
import gymnasium as gym
import numpy as np
from pprint import pprint

In [45]:
class FatiguePolicyAgent:
    '''
    Агент, имитирующий принятие решений человеком с разным уровнем усталости.
    '''
    def __init__(self, env):
        self.env = env
        self.observation_dim = env.observation_space.n
        self.actions_variants = np.array(range(env.action_space.n))  # возможные действия

        # Начальная стратегия — случайное поведение (можно трактовать как уставшее состояние)
        self.policy_probs = np.full((self.observation_dim, len(self.actions_variants)), 1.0 / len(self.actions_variants))

        # Начальные значения функции состояния
        self.state_values = np.zeros(shape=(self.observation_dim))

        # Параметры обучения
        self.max_iterations = 1000
        self.theta = 1e-6
        self.gamma = 0.99

    def policy_evaluation(self):
        '''
        Этап оценивания стратегии — насколько "эффективен" уставший или внимательный агент
        '''
        for iteration in range(self.max_iterations):
            new_state_values = np.zeros_like(self.state_values)
            for state in range(self.observation_dim):
                action_probs = self.policy_probs[state]
                state_value = 0
                for action, action_prob in enumerate(action_probs):
                    for prob, next_state, reward, done in self.env.P[state][action]:
                        state_value += action_prob * prob * (reward + self.gamma * self.state_values[next_state])
                new_state_values[state] = state_value
            # Проверка сходимости
            if np.max(np.abs(new_state_values - self.state_values)) < self.theta:
                break
            self.state_values = new_state_values
        return self.state_values

    def policy_improvement(self):
        '''
        Этап улучшения стратегии — переход от "уставшего" к "внимательному" поведению
        '''
        new_policy = np.zeros_like(self.policy_probs)
        for state in range(self.observation_dim):
            q_values = np.zeros(len(self.actions_variants))
            for action in self.actions_variants:
                for prob, next_state, reward, done in self.env.P[state][action]:
                    q_values[action] += prob * (reward + self.gamma * self.state_values[next_state])
            # Выбираем лучшее действие
            best_actions = np.where(q_values == np.max(q_values))[0]
            new_policy[state][best_actions] = 1.0 / len(best_actions)
        return new_policy

    def policy_iteration(self, iterations=20):
        '''
        Основной цикл — повторяем оценку и улучшение, пока не достигнем "устойчивого поведения"
        '''
        for i in range(iterations):
            self.policy_evaluation()
            self.policy_probs = self.policy_improvement()
        print(f'Policy Iteration завершён после {iterations} шагов.')
        from pprint import pprint




In [46]:
def play_agent(agent):
    import time
    env2 = gym.make('Taxi-v3', render_mode='human')
    state, _ = env2.reset()
    done = False
    total_reward = 0
    while not done:
        p = agent.policy_probs[state]
        action = np.random.choice(len(agent.actions_variants), p=p)
        next_state, reward, terminated, truncated, _ = env2.step(action)
        total_reward += reward
        env2.render()
        time.sleep(0.5)
        state = next_state
        done = terminated or truncated
    print(f"\nОбщий полученный reward: {total_reward}")
    env2.close()


In [49]:
from pprint import pprint
agent = FatiguePolicyAgent(env)
agent.policy_iteration(20)
pprint(agent.policy_probs)



Policy Iteration завершён после 20 шагов.
array([[0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. ],
       ...,
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. ]])


In [50]:
!pip install pygame

In [51]:
def play_agent(agent):
    '''
    Проигрывание поведения агента в консоли (без графики)
    '''
    env2 = gym.make('Taxi-v3')
    state, _ = env2.reset()
    done = False
    total_reward = 0
    steps = 0
    while not done:
        p = agent.policy_probs[state]
        action = np.random.choice(len(agent.actions_variants), p=p)
        next_state, reward, terminated, truncated, _ = env2.step(action)
        total_reward += reward
        print(f"Шаг {steps}: состояние={state}, действие={action}, награда={reward}")
        state = next_state
        steps += 1
        done = terminated or truncated
    print(f"\nОбщая награда: {total_reward}, всего шагов: {steps}")
    env2.close()


**Выводы**

В ходе выполнения лабораторной работы был реализован алгоритм итерации стратегии (Policy Iteration) для дискретной среды Taxi-v3 из библиотеки Gymnasium. Алгоритм состоял из двух ключевых этапов — оценки текущей стратегии (policy evaluation) и её улучшения (policy improvement). Для хранения вероятностной стратегии использовалась матрица вероятностей действий в каждом состоянии среды.

Особенностью данной реализации является её интерпретация в рамках магистерского проекта, посвящённого определению уровня усталости человека с использованием мобильных вычислительных устройств. Среда Taxi-v3 была условно рассмотрена как модель когнитивного поведения, где каждое состояние и действие агента может быть связано с проявлениями утомления или внимательности. Вероятностная стратегия, формируемая агентом в процессе итераций, может быть интерпретирована как динамика принятия решений пользователем с различной степенью усталости.

В результате эксперимента агент обучился принимать более рациональные действия, минимизируя количество шагов до достижения цели. Это подтверждается улучшением функции ценности состояний и сходимостью алгоритма после заданного числа итераций.

Таким образом, полученные результаты демонстрируют потенциальную применимость базовых алгоритмов обучения с подкреплением для задач моделирования и анализа поведенческих стратегий человека в условиях когнитивной нагрузки и усталости.

